In [13]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
class CombinedDataset(Dataset):
    def __init__(self, severe_ds, healthy_ds):
        self.x = torch.concat([severe_ds[0],healthy_ds[0]])
        self.y = torch.concat([severe_ds[1],healthy_ds[1]])

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [26]:
TRAIN_SET_PATH =  "C:\\Users\\rbenjos\\Desktop\\ekg_proj\\train_ds.pt"
TEST_SET_PATH =  "C:\\Users\\rbenjos\\Desktop\\ekg_proj\\test_ds.pt"

train_set = torch.load(TRAIN_SET_PATH)
test_set = torch.load(TEST_SET_PATH)

In [27]:
train_loader = DataLoader(train_set,batch_size=4,shuffle=True)
test_loader = DataLoader(test_set,batch_size=4,shuffle=True)

In [46]:
## Constants
INPUT_CH = 12
INPUT_DIM = 1250
OUTPUT_DIM = 2

class ECGNet(nn.Module):
    # a simple 1d convolutional neural network

    def __init__(self):
        super().__init__()

        # convolution layers
        self.conv1 = nn.Conv1d(INPUT_CH,INPUT_CH+2,5)
        self.conv2 = nn.Conv1d(INPUT_CH+2,INPUT_CH+4,5)
        self.conv3 = nn.Conv1d(INPUT_CH+4,INPUT_CH+6,5)
        self.conv4 = nn.Conv1d(INPUT_CH+6,INPUT_CH+8,5)
        self.conv5 = nn.Conv1d(INPUT_CH+8,INPUT_CH+10,5)
        self.conv6 = nn.Conv1d(INPUT_CH+10,INPUT_CH+12,5)

        self.conv_layers = [self.conv1,self.conv2,self.conv3,
                            self.conv4,self.conv5,self.conv6]

        # fully connected layers
        self.fc1 = nn.Linear((INPUT_CH+12)*15,180)
        self.fc2 = nn.Linear(180,60)
        self.fc3 = nn.Linear(60,20)
        self.fc4 = nn.Linear(20,2)

        # 1d max pooling
        self.maxpool = nn.MaxPool1d(2,stride=2)

    def forward(self,x):
        # convolutions
        for conv in self.conv_layers:
            x = F.relu(conv(x))
            x = self.maxpool(x)


        x = torch.flatten(x,1)
        # fully connected until 2 classes
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        x = F.softmax(x,dim=1)

        return x


In [55]:
def train_test_epoch(net,criterion,optimizer,epoch):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:
            # print and log every 10
            print(f'train_loss is {running_loss/100} at epoch {epoch+1} ')
            running_loss = 0.0

        if i % 100 == 99:
            test_epoch(net,criterion)



def test_epoch(net,criterion):
    test_loss = 0.0
    for i, data in enumerate(test_loader, 0):
        inputs, labels = data
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
    print(f'test_loss is {test_loss}')


def train_test_net(net,epochs):
    criterion = nn.BCELoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.8)

    for epoch in range(epochs):
        train_test_epoch(net,criterion,optimizer,epoch)

In [57]:
net = ECGNet()
train_test_net(net,2)

train_loss is 0.06444943904876708 at epoch 1 
train_loss is 0.06211070954799652 at epoch 1 
train_loss is 0.0595068633556366 at epoch 1 
train_loss is 0.057001038789749145 at epoch 1 
train_loss is 0.054627543091773985 at epoch 1 
train_loss is 0.052387871742248536 at epoch 1 
train_loss is 0.05026690036058426 at epoch 1 
train_loss is 0.048248887956142426 at epoch 1 
train_loss is 0.04633055627346039 at epoch 1 
train_loss is 0.04450931340456009 at epoch 1 
test_loss is 24.796063780784607
train_loss is 0.042784683406353 at epoch 1 
train_loss is 0.041144519448280334 at epoch 1 
train_loss is 0.03958672553300858 at epoch 1 
train_loss is 0.03810602605342865 at epoch 1 
train_loss is 0.03876632690429688 at epoch 1 
train_loss is 0.03547315448522568 at epoch 1 
train_loss is 0.03421928107738495 at epoch 1 
train_loss is 0.033009331524372104 at epoch 1 
train_loss is 0.03434408843517303 at epoch 1 
train_loss is 0.03595140814781189 at epoch 1 
test_loss is 18.954312473535538
train_loss is

In [58]:
for data,label in test_loader:
    output = net(data)
    print(output,label)

tensor([[0.9813, 0.0187],
        [0.9815, 0.0185],
        [0.9814, 0.0186],
        [0.9812, 0.0188]], grad_fn=<SoftmaxBackward0>) tensor([[1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.]])
tensor([[0.9812, 0.0188],
        [0.9812, 0.0188],
        [0.9811, 0.0189],
        [0.9813, 0.0187]], grad_fn=<SoftmaxBackward0>) tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]])
tensor([[0.9812, 0.0188],
        [0.9813, 0.0187],
        [0.9812, 0.0188],
        [0.9813, 0.0187]], grad_fn=<SoftmaxBackward0>) tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]])
tensor([[0.9813, 0.0187],
        [0.9815, 0.0185],
        [0.9812, 0.0188],
        [0.9813, 0.0187]], grad_fn=<SoftmaxBackward0>) tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]])
tensor([[0.9814, 0.0186],
        [0.9812, 0.0188],
        [0.9813, 0.0187],
        [0.9812, 0.0188]], grad_fn=<SoftmaxBackward0>) tensor([[1., 0.],
        [1., 0.],
       